# Structure from motion example using a video

In [2]:
# Packages
import numpy as np
import cv2 as cv

## Load video and extract grey-scale images

In [3]:
video_file = r"C:\Fotos y videos\test SfM\video1.mp4"
video = cv.VideoCapture(video_file)

delta_t = 100 # ms

In [4]:
ret = True
current_time_ms = 0

images_list = []
while ret:
    video.set(cv.CAP_PROP_POS_MSEC, current_time_ms)
    ret, frame = video.read()
    key = cv.waitKey(0)
    if key == 27 or ret is False:
        break
    cv.imshow('press esc to stop', frame)
    images_list.append(cv.cvtColor(frame, cv.COLOR_BGR2GRAY))
    current_time_ms += delta_t

cv.destroyAllWindows()

### Display loaded images

In [5]:
for im in images_list:
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Extract image descriptos

In [6]:
sift = cv.SIFT_create()
kps_des_list = []
for im in images_list:
    kps, descriptors = sift.detectAndCompute(im, None) # No mask
    kps_des_list.append({'kps': kps,
                         'des': descriptors})
    

### Display found descriptors

In [7]:
for im, des_kps in zip(images_list, kps_des_list):
    im = cv.drawKeypoints(im, des_kps['kps'], im)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Match found descriptos using k-nearest neighbours

In [8]:
# ratio of the nearest to the second nearest neighbour
ratio = 0.75

n_images = len(images_list)
matcher = cv.BFMatcher()

matches_list = []

for i in range(n_images - 1):
    good_matches = []
    matches = matcher.knnMatch(kps_des_list[i]['des'], kps_des_list[i+1]['des'], k=2)
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_matches.append(m)
    matches_list.append(good_matches)

### display good matches

In [9]:
for i, matches in enumerate(matches_list):
    im = cv.drawMatches(images_list[i], kps_des_list[i]['kps'], images_list[i+1], kps_des_list[i+1]['kps'], matches, im, 
                        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Fundamental matrices estimation using RANSAC

In [38]:
reproj_fundmat_masks_list = []
masked_matches_list = []

for i in range(n_images - 1):
    src_points = np.array([kps_des_list[i]['kps'][m.queryIdx].pt for m in matches_list[i]], dtype='float32')
    dst_points = np.array([kps_des_list[i+1]['kps'][m.trainIdx].pt for m in matches_list[i]], dtype='float32') 
    fundamental_matrix, mask = cv.findFundamentalMat(src_points, dst_points, method=cv.FM_RANSAC, ransacReprojThreshold=.01)
    reproj_fundmat_masks_list.append({"fundamental_matrix": fundamental_matrix, 
                                      "mask": mask})
    # mask matches from reprojection error
    masked_matches = []
    for i_match in range(len(matches_list[i])):
        if mask[i_match] == 1:
            masked_matches.append(matches_list[i][i_match])
    masked_matches_list.append(masked_matches)


### Display matches after removing outliers

In [39]:
for i, matches in enumerate(matches_list):            
    im = cv.drawMatches(images_list[i], kps_des_list[i]['kps'], images_list[i+1], kps_des_list[i+1]['kps'], 
                        masked_matches_list[i], 
                        im, 
                        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()